In [8]:
######## 데이터세트 클래스 선언

import os
import torch
from PIL import Image
from pycocotools.coco import COCO
from torch.utils.data import Dataset

# MS COCO 데이터세트는 80개의 클래스로 이루어짐
# 이번 절에서는 개와 고양이 클래스를 소규모로 샘플랑해 실습 진행
class COCODataset(Dataset):
    def __init__(self, root, train, transform=None):
        super().__init__()
        directory = "train" if train else "val"
        # annotation json 파일 경로 설정
        annotations = os.path.join(root, "annotations", f"{directory}_annotations.json")
        
        # 복잡한 주석 정보 쉽게 다룰 수 있도록 설계된 COCO 클래스
        self.coco       = COCO(annotations)
        self.iamge_path = os.path.join(root, directory)
        self.transform  = transform

        self.categories = self._get_categories()
        self.data       = self._load_data()

    def _get_categories(self):
        categories = {0: "background"}
        # cats(categories) 속성에서 카테고리 정보 불러오기
        for category in self.coco.cats.values():
            categories[category["id"]] = category["name"]
        return categories
    
    ### COCO 데이터세트 불러오기
    def _load_data(self):
        data = []
        for _id in self.coco.imgs:
            # 이미지 정보 로드
            file_name  = self.coco.loadImgs(_id)[0]["file_name"]
            # Output: [{'license': 1, 'file_name': '000000495357.jpg', ..., 'id': 495357}]이므로 [0]처리
            image_path = os.path.join(self.iamge_path, file_name)
            image      = Image.open(image_path).convert("RGB")

            # 주석 정보 로드 (카테고리 id, bbox 좌표)
            boxes  = []
            labels = []
            # getAnnIds: 이미지 id 입력받고 어노테이션 id 반환
            anns   = self.coco.loadAnns(self.coco.getAnnIds(_id))
            for ann in anns:
                # x, y는 왼쪽 상단 모서리의 좌표
                x, y, w, h = ann["bbox"]
                
                boxes.append([x, y, x + w, y + h])
                labels.append(ann["category_id"])   # [1,2,1,1,..]

            # 타겟 딕셔너리 생성
            target = {
            "image_id": torch.LongTensor([_id]),
            "boxes"   : torch.FloatTensor(boxes),
            "labels"  : torch.LongTensor(labels)
            }
            # 데이터 리스트에 추가
            data.append([image, target])
        return data

    ### 호출 및 길이 반환 메서드
    def __getitem__(self, index):
        image, target = self.data[index]
        if self.transform:
            image = self.transform(image)
        return image, target

    def __len__(self):
        return len(self.data)

In [9]:
####### 데이터로더
from torchvision import transforms
from torch.utils.data import DataLoader


def collator(batch):
    return tuple(zip(*batch))

# PIL to float
transform = transforms.Compose(
    [
        transforms.PILToTensor(),
        transforms.ConvertImageDtype(dtype=torch.float)
    ]
)

train_dataset = COCODataset("../datasets/coco", train=True, transform=transform)
test_dataset  = COCODataset("../datasets/coco", train=False, transform=transform)

train_dataloader = DataLoader(
    train_dataset, batch_size=4, shuffle=True, drop_last=True, collate_fn=collator
)
test_dataloader  = DataLoader(
    test_dataset, batch_size=1, shuffle=True, drop_last=True, collate_fn=collator
)

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [10]:
######## 백본 및 모델 구조 정의
from torchvision import models
from torchvision import ops
from torchvision.models.detection import rpn
from torchvision.models.detection import FasterRCNN


backbone = models.vgg16(weights="VGG16_Weights.IMAGENET1K_V1").features
backbone.out_channels = 512

# 영역 제안 네트워크 (관심 영역 후보군 output)
anchor_generator = rpn.AnchorGenerator(
    sizes         = ((32, 64, 128, 256, 512),),
    aspect_ratios = ((0.5, 1.0, 2.0),)
)

# 관심 영역 풀링 (관심 영역 후보군 input, 고정된 크기의 관심영역 특징맵 output)
roi_pooler = ops.MultiScaleRoIAlign(
    featmap_names  = ["0"], # 각 이름에 해당하는 특징 맵 입력 (0: VGG-16 특징추출 계층)
    output_size    = (7, 7),# 특징 맵 output size
    sampling_ratio = 2      # roi align에서 샘플링에 사용되는 특징맵
)

device = "mps" if torch.backends.mps.is_available() and torch.backends.mps.is_built() else "cpu"
model  = FasterRCNN(
    backbone             = backbone,
    num_classes          = 3,   # 배경, cat, dog
    rpn_anchor_generator = anchor_generator,
    box_roi_pool         = roi_pooler
).to(device)

In [11]:
######## 최적화 함수 및 학습률 스케쥴러
from torch import optim

# 학습이 가능한 매개변수
params       = [p for p in model.parameters() if p.requires_grad]
optimizer    = optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)
# 5step마다 학습률 0.1씩 감소
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
### 임의 추가 (해결 x)
import os
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

In [ ]:
######## Faster R-CNN 미세 조정
for epoch in range(5):
    cost = 0.0
    for idx, (images, targets) in enumerate(train_dataloader):
        images  = list(image.to(device) for image in images)
        # target에 image_id, boxes(박스회귀), labels(박스분류) ground truth 담겨있음
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        # 분류손실, 회귀손실, 객체 유무손실, 영역제안 네트워크 손실 반환
        loss_dict = model(images, targets)
        losses    = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        cost += losses

    lr_scheduler.step()
    cost = cost / len(train_dataloader)
    print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

![](../미세조정.jpeg)

In [ ]:
######## 모델 추론 및 시각화
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from torchvision.transforms.functional import to_pil_image

# 사각형과 텍스트를 이미지 위에 그리는 함수
def draw_bbox(ax, box, text, color): 
    ax.add_patch(
        plt.Rectangle(
            # 0,1: 왼쪽위 모서리 x,y / 2,3: 오른쪽아래 모서리 x,y
            xy        = (box[0], box[1]),
            width     = box[2] - box[0],
            height    = box[3] - box[1],
            fill      = False,
            edgecolor = color,
            linewidth = 2,
        )
    )
    ax.annotate(
        text     = text,
        xy       = (box[0] - 5, box[1] - 5),
        color    = color,
        weight   = "bold",
        fontsize = 13,
    )

threshold  = 0.5    # 신뢰도 임계값
categories = test_dataset.categories
with torch.no_grad(): 
    model.eval()
    for images, targets in test_dataloader: 
        images  = [image.to(device) for image in images]
        outputs = model(images)
        
        # 0.5 이상의 신뢰도 가진 것만 필터링
        boxes  = outputs[0]["boxes"].to("cpu").numpy()
        labels = outputs[0]["labels"].to("cpu").numpy()
        scores = outputs[0]["scores"].to("cpu").numpy()
        
        boxes  = boxes[scores >= threshold].astype(np.int32)
        labels = labels[scores >= threshold]
        scores = scores[scores >= threshold]

        fig = plt.figure(figsize=(8, 8))
        ax  = fig.add_subplot(1, 1, 1)
        plt.imshow(to_pil_image(images[0]))

        # 예측 상자 시각화
        for box, label, score in zip(boxes, labels, scores): 
            draw_bbox(ax, box, f"{categories[label]} - {score:.4f}", "red")

        # 실제 객체 상자 시각화
        tboxes  = targets[0]["boxes"].numpy()
        tlabels = targets[0]["labels"].numpy()
        for box, label in zip(tboxes, tlabels): 
            draw_bbox(ax, box, f"{categories[label]}", "blue")
            
        plt.show()

![](../모델추론.jpeg)

In [ ]:
######## 모델 평가
import numpy as np
from pycocotools.cocoeval import COCOeval


with torch.no_grad(): 
    model.eval()
    coco_detections = []
    for images, targets in test_dataloader: 
        images  = [img.to(device) for img in images]
        outputs = model(images)
        
        # 각 이미지의 타겟(boxes,scores,labels)정보 저장
        for i in range(len(targets)): 
            image_id    = targets[i]["image_id"].data.cpu().numpy().tolist()[0]
            boxes       = outputs[i]["boxes"].data.cpu().numpy()
            boxes[:, 2] = boxes[:, 2] - boxes[:, 0] # w
            boxes[:, 3] = boxes[:, 3] - boxes[:, 1] # h
            scores      = outputs[i]["scores"].data.cpu().numpy()
            labels      = outputs[i]["labels"].data.cpu().numpy()

            # 한 이미지에서 경계 상자(boxes) 여러개인 경우
            for instance_id in range(len(boxes)): 
                box        = boxes[instance_id, :].tolist()
                prediction = np.array(
                    [
                        image_id,
                        box[0],
                        box[1],
                        box[2],
                        box[3],
                        float(scores[instance_id]),
                        int(labels[instance_id]),
                    ]
                )
                coco_detections.append(prediction)

    # detection 결과 저장
    coco_detections = np.asarray(coco_detections)
    # 데이터셋 groundtruth API
    coco_gt         = test_dataloader.dataset.coco      # 데이터셋의 ground truth
    coco_dt         = coco_gt.loadRes(coco_detections)  # detection 결과를 COCO 데이터구조로 로드
    coco_evaluator  = COCOeval(coco_gt, coco_dt, iouType="bbox") # 모델 평가 클래스
    # 모델 평가
    coco_evaluator.evaluate()
    coco_evaluator.accumulate()
    coco_evaluator.summarize()

IoU 임계값과 area(객체의 크기), maxDets(검출할 수 있는 최대 객체수)에 따른 모델의 성능 (높을수록 좋은 성능)

![](../모델평가.jpeg)